In [591]:
import requests
import json
from datetime import datetime

'''
First step: Check The top players in our region of interest. Due to the growth of the servers, the bigger ones with the 
best players are: EUW, Korea, China and NA.

China is barely independent, so accessing to the servers is not allowed. In its place, we'll use Nordic europe.

In order to work properly, we'll define a structure that englobes servers paths. Then, just add the api key to all of them. 
Remember this api Key needs to be updated each day (at least for now).
'''

# First 
ServerList = ["euw1", "kr", "na1", "eun1"]

path1 = "https://"
path2 = ".api.riotgames.com/tft/league/v1/"

LeagueList = ["challenger", "grandmaster", "master"]

path3 = "?api_key="

# Necessary to update the key periodically. Keep in mind it's also necessary to register into: https://developer.riotgames.com/
Key = "RGAPI-083941ff-2f77-445c-b5c9-633db71fa541"

# Building request
'''
ServerList 0 - euw
ServerList 1 - kr
ServerList 2 - na
ServerList 3 - eun

LeagueList 0 - challenger
LeagueList 1 - grandmaster
LeagueList 2 - master
'''

server = ServerList[0]
league = LeagueList[0]

output = path1+server+path2+league+path3+Key
print(output)

RankPlayers = requests.get(output) 
RankPlayers # Api request response: 200 it's ok!

https://euw1.api.riotgames.com/tft/league/v1/challenger?api_key=RGAPI-083941ff-2f77-445c-b5c9-633db71fa541


<Response [200]>

In [592]:
League = RankPlayers.json()

# Selecting keys to iterate
LeagueKeys = League.keys()

# Once we have keys stored, based in the structure, entries is formed by an array of Json structures where each element is a player
PlayersJson = League["entries"]

# PlayersJson is an array made by json structures we only want to store summoner's name, league points, currently rank and winrate
PlayersKeys = PlayersJson[0].keys()

PlayerPoints = []

# Create Json structure
for players in PlayersJson:
    Json = {"summonerId":players["summonerId"],"summonerName":players["summonerName"],"leaguePoints":players["leaguePoints"]}
    PlayerPoints.append(Json)

# Sort by league points (better players to the top)
PlayerPoints = sorted(PlayerPoints, key=lambda k: k['leaguePoints'], reverse=True)

#Visualize how is the data stored & how many players are on the selected league
print(len(PlayerPoints))
PlayerPoints[0:5]

200


[{'summonerId': '55CccEiSqobwpiCV1hHSun5A98k2XCThWPo-Eo61o3O1xPM2',
  'summonerName': 'Deis1k',
  'leaguePoints': 1607},
 {'summonerId': 'y99lozZldF7ZaOkfp4iND5E_EerDVwczuOLG__xdyjEL_Jnk',
  'summonerName': 'Monphis',
  'leaguePoints': 1524},
 {'summonerId': 'NJEizsEvtDoyflx6NltpbNgduI8lLA2ybCSbCsOWnlqkrSfx',
  'summonerName': 'Voltariusx',
  'leaguePoints': 1490},
 {'summonerId': 'eeGo2TixBlpJ-Mbb_CmDut-krarEt_q2E5pR-wUR7oKpkSRV',
  'summonerName': 'Ukkkko',
  'leaguePoints': 1486},
 {'summonerId': '87_x372yfGQfL6SfJbCbMpiPw2EWMqGK5FIRlzehbsnIbXA',
  'summonerName': 'DarkHydra',
  'leaguePoints': 1484}]

In [593]:
# Time to select the "top players" in order to find their games
upperLimit = 20
lowerLimit = 0

topPlayers = PlayerPoints[lowerLimit:upperLimit]

print(len(topPlayers))
# 1st step: use summoner ID to find out PUUID using request API in TFT-SUMMONER-V1, adding it to this own structure

import time

PUUIDpath2 = ".api.riotgames.com/tft/summoner/v1/summoners/"

puuid = []
'''
Finding loop for all PUUID data, each loop is for each player, we just need to maintain that PUUID. Finally, jsut 
request that PUUID game.

'''


for player in topPlayers:
    # PUUID API request for each summoner
    PUUIDOutput = path1+server+PUUIDpath2+player["summonerId"]+path3+Key 
    # API request
    PUUIDRequest = requests.get(PUUIDOutput) 
    print(PUUIDRequest)
    jsonStructure = json.loads(PUUIDRequest.text)
    puuid.append(jsonStructure["puuid"])

20
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [594]:
len(puuid)

20

In [595]:
'''
This puuid is from top elo players. Now, we need to request games played by this Puuid and store them into a new list.
this request is "splitted" not in servers but in region. It's necessary to filter which region is.
'''

matchPath2 = ".api.riotgames.com/tft/match/v1/matches/by-puuid/"
# we define the amount of games here [actually 20 for testing]
matchPath3 = "/ids?count=20&api_key="


if (server == ServerList[0] or server == ServerList[3]):
    region = "europe"
elif (server == ServerList[1]):
    region = "asia"
else:
    region = "americas"
    
# Now we have region selected, we can build out api request for finding matches

matchesId = []

for player in puuid:
    matchPath = path1+region+matchPath2+player+matchPath3+Key
    
    # Request
    matchRequest = requests.get(matchPath) 
    matchesId.append(matchRequest.text)
    time.sleep(1)


gamesId = []

print("games Id request")
# matchesId is formed by 5 strings (one for each request), it's necessary to clean it
for matchId in matchesId: 
    # erase [] from the string
    matchId = matchId[1:-1] 
    # transform from string to list
    matchId = matchId.split(",") #
    # iterate in order to obtain all matches Id in the same list
    for realMatchId in matchId:
        gamesId.append(realMatchId[1:-1])
    time.sleep(1)

# Now with the data stored, it's time to request data for each game

# Start the loops iterating for each game, request all that info. Once that info is properly stored, create dataframe.
gameInfoPath2 = ".api.riotgames.com/tft/match/v1/matches/"
gameInfo = []

print("game info request")
for mId in gamesId:
    # Create the path
    gameInfoPath = path1+region+gameInfoPath2+mId+path3+Key
    
    # Create request
    gameRequest = requests.get(gameInfoPath)
    
    # Extract information about game request, jsonload to add it directly
    gameInfo.append(json.loads(gameRequest.text))
    
    time.sleep(1)


games Id request
game info request


In [596]:
print(len(gameInfo))

400


In [597]:
# Initialize the database
import pandas as pd

dataf=pd.DataFrame(columns = ["matchId","hour","day","month","release","playerPuuid","LastRoundAlive","playerEndingGame",
                             "EndGameTime", "Ctype1","TierC1","Ctype2","TierC2","Ctype3","TierC3","Ctype4","TierC4",
                              "Ctype5","TierC5","Ctype6","TierC6","Ctype7","TierC7","Ctype8","TierC8"])

In [603]:
# gameInfo is a list containing al the game data, consider each element has all the information from a single game
# This data is the one we'll be using to process, we need to filter it in order to obtain those features 



gameInfoFiltered = []
for game in gameInfo:
    # From metadata key we only want to store game Id
    matchId =game["metadata"]["match_id"] 
    # Date information
    timestamp = int(game["info"]["game_datetime"]/1000) 
    dt_object = datetime.fromtimestamp(timestamp)
    day = dt_object.day
    month = dt_object.month
    hour = dt_object.hour
    
    release = game["info"]["game_version"]
    releaseValue = []
    
    for i in range(0,len(release)):
        if release[i] == "<":
            index = i
            break

    # now we found the starting index, just select what's behind that
    releaseValue = str(release[index+10:len(release)-1])
    
    # Time to store each player (PUUID) with the following information: last round, level, time eliminated + composition with level
    for player in game["info"]["participants"]:
        puuid = player["puuid"]
        playerLastRound = player["last_round"]
        playerEndingGame = player["placement"]
        EndGameTime = player["time_eliminated"]
        

        
        # Compositions 
        compo = []
        for comp in player["traits"]: # Iterate through all 
            midProcess = ["-","-"] # Default value
            if comp["tier_current"] > 0:
                midProcess = [comp["name"], comp["tier_current"]]
                compo.append(midProcess)        
        
        while (len(compo)<8):
            compo.append(["-","-"])

            
        # Add compositions, would be nice to filter instead copy-paste. Moreover, it's necessary some process as long as 
        # might be highly unlikely to get 10 
        
        df2 = pd.DataFrame({"matchId":[matchId], "hour":[hour], "day":[day], "month":[month], "release":[releaseValue],
                       "playerPuuid":[puuid], "LastRoundAlive":[playerLastRound], "playerEndingGame":[playerEndingGame],
                       "EndGameTime":[EndGameTime],"Ctype1":[compo[0][0]], "TierC1":[compo[0][1]],"Ctype2":[compo[1][0]], "TierC2":[compo[1][1]],
                       "Ctype3":[compo[2][0]], "TierC3":[compo[2][1]], "Ctype4":[compo[3][0]], "TierC4":[compo[3][1]],
                       "Ctype5":[compo[4][0]], "TierC5":[compo[4][1]], "Ctype6":[compo[5][0]], "TierC6":[compo[5][1]],
                       "Ctype7":[compo[6][0]], "TierC7":[compo[6][1]], "Ctype8":[compo[7][0]], "TierC8":[compo[7][1]]})
        

        dataf = dataf.append(df2) #ignore_index = True // removed


        
# The following code is for further steps analyzing champion-item performance.
''' 
   # Champions
    champAndItems = []
    for champ in player["units"]:
        midProcess = {"champion":"-","tier":"-", "item1":"-", "item2":"-", "item3":"-"} # Default value
        midProcess["champion"] = champ["character_id"]
        midProcess["tier"] = champ["tier"]
        nItems = len(champ["items"])
        if nItems >=1:
            midProcess["item1"] = champ["items"][0]
            if nItems >= 2:
                midProcess["item2"] = champ["items"][1]
                if nItems == 3:
                    midProcess["item3"] = champ["items"][2]
                
    champAndItems.append(midProcess)
    
'''


' \n   # Champions\n    champAndItems = []\n    for champ in player["units"]:\n        midProcess = {"champion":"-","tier":"-", "item1":"-", "item2":"-", "item3":"-"} # Default value\n        midProcess["champion"] = champ["character_id"]\n        midProcess["tier"] = champ["tier"]\n        nItems = len(champ["items"])\n        if nItems >=1:\n            midProcess["item1"] = champ["items"][0]\n            if nItems >= 2:\n                midProcess["item2"] = champ["items"][1]\n                if nItems == 3:\n                    midProcess["item3"] = champ["items"][2]\n                \n    champAndItems.append(midProcess)\n    \n'

In [604]:
dataf.head(10)

,matchId,hour,day,month,release,playerPuuid,LastRoundAlive,playerEndingGame,EndGameTime,Ctype1,...,Ctype4,TierC4,Ctype5,TierC5,Ctype6,TierC6,Ctype7,TierC7,Ctype8,TierC8
0,EUW1_4627054925,20,27,5,10.10,-3to15uUjeOneXycYfZw2aDG4QMWyhMIuJIzIWQpBwLchH...,30,6,1688.711304,Blaster,...,-,-,-,-,-,-,-,-,-,-
0,EUW1_4627054925,20,27,5,10.10,1FmO7Husl99J9_MrAGQMlkLaFv4c1xCsKU6t6N69RNsFkg...,34,3,1926.799561,Chrono,...,Set3_Blademaster,1,Set3_Celestial,1,Set3_Mystic,1,-,-,-,-
0,EUW1_4627054925,20,27,5,10.10,MQvp6_GJH_jIWSO9gkRt4ug7MewLZyRmScbwF579Wz7cIR...,35,1,1937.986938,Chrono,...,Set3_Mystic,1,Sniper,1,-,-,-,-,-,-
0,EUW1_4627054925,20,27,5,10.10,N0sSGfAvZ69cBVM2W-ms2tsszL8vwVeiCOE44sbOiuve1N...,33,4,1852.317139,Chrono,...,-,-,-,-,-,-,-,-,-,-
0,EUW1_4627054925,20,27,5,10.10,Kf_uyA2ZyVuCo3v840kQk7uOJs7b-OopScMRJl_e949p42...,31,5,1749.444580,Chrono,...,Set3_Blademaster,1,Set3_Celestial,1,Set3_Mystic,1,Valkyrie,1,-,-
0,EUW1_4627054925,20,27,5,10.10,LRofJE9K1TPvWWPqwa2FIYqfofZy2KU7Bf_T3hZqSeJWIM...,35,2,1937.986938,ManaReaver,...,SpacePirate,1,Vanguard,1,-,-,-,-,-,-
0,EUW1_4627054925,20,27,5,10.10,oWgSj5spUlyct3SutH3nRnhTC_Y32UCqkXPH3XZ-JuHBqH...,28,8,1592.785034,Chrono,...,Mercenary,1,Set3_Brawler,1,Set3_Sorcerer,1,Set3_Void,1,-,-
0,EUW1_4627054925,20,27,5,10.10,sGoE01iKfd3SDoBrrCHC5iFVpwbu60TcrhhOzbhIupZICL...,28,7,1596.177368,Demolitionist,...,Sniper,1,SpacePirate,1,Vanguard,1,-,-,-,-
0,EUW1_4626869461,20,27,5,10.10,MQvp6_GJH_jIWSO9gkRt4ug7MewLZyRmScbwF579Wz7cIR...,35,4,1929.479858,Blaster,...,Rebel,2,Set3_Mystic,1,-,-,-,-,-,-
0,EUW1_4626869461,20,27,5,10.10,r5RH-gSmHbNX1AXgESrItsyjzuoQh_zMfpC_O1Ue2Q0q_s...,37,3,2018.850220,Infiltrator,...,Valkyrie,1,-,-,-,-,-,-,-,-


In [606]:
'''
Games could be repeated in case two players played together in the same game, but considering the amount of unique values
within global requests (20 players, 20 games) greater than 90% of games aren't repeated. We'll assume this values won't affect 
that much results.
'''
len(dataf.matchId.unique())/400

0.9425

In [607]:
# Time to store our dataset like as it is right now
dataf.to_csv("C:/Users\Sergi\Desktop\TFT.csv")
# This is my own path !